In [1]:
# 当运行一个耗时较长的训练过程时， 最佳的做法是定期保存中间结果， 以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。 
# 因此，现在是时候学习如何加载和存储权重向量和整个模型了。
# torch.rand(size) # 均匀分布U(0, 1)
# torch.randn(size) # 高斯正太, random normal
# torch.normal(mean, std, size)
# torch.arange(start, end, step)
# torch.FloatTensor([]) 创建一个float32的张量
# pytorch 框架名称
# torch 顶层python包
# torch.nn 神经网络的模块
# torch.nn.init 模块下的子模块,用来初始化函数,不可实例化,用来组织代码
# torch.Linear 类，需要实例化,用来生成对象

import torch
from torch import nn
from torch.nn import functional as F

In [2]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [3]:
# 我们现在可以将存储在文件中的数据读回内存
# 这里解释一下pickle的机制与用法
# pickle 是 Python 自带的“序列化（serialization）工具”，用来把 Python 对象转成二进制数据，然后保存到文件或通过网络传输。
# python 中的 save(obj, 'file'), 在底层其实就是 pickle.dump(obj, file)
# python 中的 load('file'), 在底层就是 pickle.load(file)
# 但是问题在于，pickle会执行任意代码, 会导致风险

In [4]:
x2 = torch.load('x-file', weights_only=True)
x2

tensor([0, 1, 2, 3])

In [5]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files', weights_only=False)
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [6]:
# 我们甚至可以写入或读取从字符串映射到张量的字典。 当我们要读取或写入模型中的所有权重时，这很方便

In [7]:
mydict = {'x': x, 'y': y} # 字符串映射到tensor
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict', weights_only = False)
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [8]:
# 存单个权重向量（或其他张量）确实有用， 但是如果我们想保存整个模型，并在以后加载它们， 单独保存每个向量则会变得很麻烦
# 因此，深度学习框架提供了内置函数来保存和加载整个网络。 需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。 
# 例如，如果我们有一个3层多层感知机，我们需要单独指定架构。 因为模型本身可以包含任意代码，所以模型本身难以序列化。(!) 
# 因此，为了恢复模型，我们需要用代码生成架构， 然后从磁盘加载参数。 让我们从熟悉的多层感知机开始尝试一下

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size = (2, 20))
Y = net(X)

In [10]:
torch.save(net.state_dict(), 'mlp.params')

In [11]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params', weights_only=True))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [12]:
# 由于两个实例具有相同的模型参数，在输入相同的X时， 两个实例的计算结果应该相同。 让我们来验证一下。

In [13]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [14]:
!nvidia-smi

Mon Nov 24 14:14:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.42                 Driver Version: 581.42         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   47C    P0            N/A  /  130W |    6667MiB /   8188MiB |      7%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
import sys
print(sys.executable)


D:\anaconda3\envs\systemapp\python.exe


In [16]:
torch.cuda.device_count()

1